In [ ]:
# Set seeds
from numpy.random import seed
import tensorflow as tf
seed(42)
tf.random.set_seed(42)

In [ ]:
# Upgrade module imbalanced-learn to run on Google Colab
#!pip install imbalanced-learn --upgrade

# Import modules
import imblearn
import keras
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import sklearn

from google.colab import drive
from imblearn.over_sampling import RandomOverSampler
from keras.applications.resnet import ResNet50
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, History
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.models import Sequential,Model,load_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from platform import python_version
from sklearn.model_selection import train_test_split

%matplotlib inline

# Versions
print("Version Python:",python_version())
print()
print("Version Imbalanced-learn:",imblearn.__version__)
print("Version Keras:",keras.__version__)
print("Version Matplotlib:",matplotlib.__version__)
print("Version NumPy:",np.__version__)
print("Version Pandas:",pd.__version__)
print("Version Scikit-learn:",sklearn.__version__)
print("Version TensorFlow:",tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set data folder and read data
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/'
data = pd.read_pickle(os.path.join(data_folder, 'data.pkl'))
data = data[data["age_group"] == 'Older']

In [ ]:
# Setting data augmentation parameters 
data_augmentation = ImageDataGenerator(
    rotation_range = 60,
    zoom_range = 0.2,  
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,  
    vertical_flip = True,  
    shear_range = 10) 

In [ ]:
# Create training, validation amd test sets (Derived from: https://www.kaggle.com/kaimingk/skin-cancer-mnist-ham10000)

# Split 70%, 10%, 20%. Test and validation, together 30%, should be from unique ID's only
size_val_test = 0.3 * len(data)
val_test_ratio_from_unique = size_val_test/len(data[data.unique == True])

# Training data = remainder of unique ID's + non-uniques. Split is stratified by lesion type (benign/malignant)
bb_data_train_unique, bb_data_val_test = train_test_split(data[data["unique"] == True], test_size = val_test_ratio_from_unique, stratify=data[data["unique"] == True]["lesion_type_cat"], random_state = 42)
bb_data_train = pd.concat((bb_data_train_unique, data[data["unique"] == False]), axis = 0)

# Split validation and test sets. Split is stratified by lesion type (benign/malignant)
bb_data_validation, bb_data_test = train_test_split(bb_data_val_test, test_size = 0.6667, stratify=bb_data_val_test["lesion_type_cat"], random_state = 42)

# Class balancing (random oversampling)
X = bb_data_train.drop(['lesion_type_cat'], axis=1)
y = bb_data_train['lesion_type_cat']

X_test = bb_data_test.drop(['lesion_type_cat'], axis=1)
y_test = bb_data_test['lesion_type_cat']

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

ros = RandomOverSampler(random_state=42)
X_test_resampled, y_test_resampled = ros.fit_resample(X_test, y_test)

bb_data_train = pd.concat([X_resampled, y_resampled], axis=1)
bb_data_test = pd.concat([X_test_resampled, y_test_resampled], axis=1)

# Create variables for feature (x) and target (y) training, validation and test sets
bb_data_train_x = np.asarray(bb_data_train['image'].tolist())
bb_data_validation_x = np.asarray(bb_data_validation['image'].tolist())
bb_data_test_x = np.asarray(bb_data_test['image'].tolist())

bb_data_train_y = np.asarray(bb_data_train['lesion_type_cat'].tolist())
bb_data_validation_y = np.asarray(bb_data_validation['lesion_type_cat'].tolist())
bb_data_test_y = np.asarray(bb_data_test['lesion_type_cat'].tolist())

# One-hot encoding of target variable
bb_num_classes = len(np.sort(data['lesion_type_cat'].unique()))

bb_data_train_y = to_categorical(bb_data_train_y, num_classes = bb_num_classes)
bb_data_validation_y = to_categorical(bb_data_validation_y, num_classes = bb_num_classes)
bb_data_test_y = to_categorical(bb_data_test_y, num_classes = bb_num_classes)

In [ ]:
# Fit data augmentation
data_augmentation.fit(bb_data_train_x)

In [ ]:
# Creating ResNet model (Derived from: https://www.kaggle.com/jnegrini/ham10000-analysis-and-model-comparison)

# Model parameters
input_shape = (75, 100, 3)
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
epochs = 50
batch_size = 32

# Callbacks
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5, verbose=1, factor=0.5, min_lr=0.00001)
early_stopping_monitor = EarlyStopping(patience=20,monitor='val_accuracy')
history = History()

# Define model architecture
base_model = ResNet50(include_top=False, input_shape=(75,100, 3),pooling = 'avg', weights = 'imagenet');

ResNet50model_binary = Sequential()
ResNet50model_binary.add(base_model)
ResNet50model_binary.add(Dropout(0.2))
ResNet50model_binary.add(Dense(128, activation="relu"))
ResNet50model_binary.add(Dropout(0.2))
ResNet50model_binary.add(Dense(bb_num_classes, activation = 'softmax'))
ResNet50model_binary.summary()

# Layers in ResNet are pretrained (ImageNet)
for layer in base_model.layers:
    layer.trainable = False

# Compile model
ResNet50model_binary.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 23,850,242
Trainable params: 23,797,122
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
# Fit and save model 
bb_history = ResNet50model_binary.fit(data_augmentation.flow(bb_data_train_x,bb_data_train_y, batch_size=batch_size),
                        epochs = epochs, validation_data = (bb_data_validation_x,bb_data_validation_y),
                        verbose = 1, steps_per_epoch=bb_data_train_x.shape[0] // batch_size, 
                        callbacks=[learning_rate_reduction,early_stopping_monitor, history])

ResNet50model_binary.save(os.path.join(data_folder,"Age_Old_binary_model"))

Epoch 1/50
165/165 [==============================] - 204s 1s/step - loss: 0.7342 - accuracy: 0.6203 - val_loss: 0.3119 - val_accuracy: 0.8255
Epoch 2/50
165/165 [==============================] - 171s 1s/step - loss: 0.5602 - accuracy: 0.6974 - val_loss: 0.2672 - val_accuracy: 0.8621
Epoch 3/50
165/165 [==============================] - 171s 1s/step - loss: 0.5375 - accuracy: 0.7127 - val_loss: 0.2930 - val_accuracy: 0.8290
Epoch 4/50
165/165 [==============================] - 172s 1s/step - loss: 0.5348 - accuracy: 0.7146 - val_loss: 0.3054 - val_accuracy: 0.8412
Epoch 5/50
119/165 [====================>.........] - ETA: 43s - loss: 0.5161 - accuracy: 0.7283

In [ ]:
# Accuracy over epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Loss - Age_Old (binary)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
filename = str('Model Accuracy - Age_Old (binary).png')
plt.savefig(os.path.join(data_folder,filename), dpi=600)
plt.show()

# Loss over epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - Age_Old (binary)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
filename = str('Model Loss - Age_Old (binary).png')
plt.savefig(os.path.join(data_folder,filename), dpi=600)
plt.show()


In [ ]:
with open(os.path.join(data_folder, 'data_bb_age_old.pkl'), 'wb') as f:
    pickle.dump([bb_data_train_x,bb_data_validation_x,bb_data_test_x,bb_data_train_y,bb_data_validation_y,bb_data_test_y], f)